In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_classic.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_classic.storage import LocalFileStore
from langchain_core.runnables import RunnablePassthrough

cache_dir = LocalFileStore("./.cache/")
chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[StreamingStdOutCallbackHandler()]
)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=600,
    chunk_overlap=100,
    separator="\n"
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir, key_encoder="sha256"
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
        ("human", "{question}"),
    ]
)

chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | chat
chain.invoke("Describe Victory Mansions")


AIMessage(content='Victory Mansions is a building where Winston Smith resides. It has glass doors that let in gritty dust, and the hallway smells of boiled cabbage and old rag mats. Inside the flat, there is a telescreen on the wall that cannot be completely shut off. The building is run-down, with a faulty lift and posters of an enormous face with the caption "BIG BROTHER IS WATCHING YOU."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 1936, 'total_tokens': 2019, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CdXeCu2negO1AEmFh9qbGNQyEZ5Es', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--8daae862-52d9-4ffb-8e44-3a3fe5f82

In [ ]:
results = vectorstore.similarity_search("where does Winston live")
results